In [15]:
# -*- coding: utf-8 -*-
"""XGBoostWB_Traffic"""

import sys
#Import Libraries
#import tensorflow as tf
import pandas as pd
import numpy as np
import os
#import matplotlib
#import matplotlib.pyplot as plt
import random
# %matplotlib inline
import shutil
from random import shuffle
random.seed(42) 
import itertools

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from datetime import datetime, timedelta
from sklearn import preprocessing

%load_ext autoreload
%autoreload 2
from GBRT_for_TSF.utils import *
from mpmf.utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
### Change here!!!

include_features = True
include_motif_information = False

# for top-k motifs
# import warnings

# warnings.filterwarnings("ignore")
###

num_periods_output = 24  # to predict
num_periods_input = 24  # input

ALL_Test_Data = []
ALL_Test_Prediction = []

file_name = "traffic.npy"
data_path = r"../data/" + file_name
data = np.load(data_path)
### Change here!!!

# data = data[0:90, :]
data = data[0:1, :]

###
data = pd.DataFrame(data)

In [17]:
"""## preprocessing"""


def New_preprocessing(TimeSeries):
    Data = []
    start_date = datetime(2012, 1, 1, 00, 00, 00)  # define start date
    for i in range(0, len(TimeSeries)):
        record = []
        record.append(TimeSeries[i])
        record.append(start_date.month)
        record.append(start_date.day)
        record.append(start_date.hour)
        record.append(start_date.weekday())
        record.append(start_date.timetuple().tm_yday)
        record.append(start_date.isocalendar()[1])
        start_date = start_date + timedelta(hours=1)
        Data.append(record)
    ########## change list of lists to df ################

    if include_motif_information:
        ### Change here!!!

        # Top-K Motifs (Average)

        # df_motif = get_top_k_motifs(
        #     TimeSeries, num_periods_output, k=3, l=5, include_itself=True
        # )
        # interested_features = [c for c in df_motif.columns if ("idx" not in c)]
        # df_motif = df_motif[interested_features]
        # # df_motif = compute_point_after_average(df_motif)
        # df_motif = compute_point_after_average(df_motif, method="weighted")

        # Top-K Motifs (Direct)

        # df_motif = get_top_k_motifs(
        #     TimeSeries, num_periods_output, k=3, l=5, include_itself=True
        # )
        # interested_features = [
        #     c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
        # ]
        # df_motif = df_motif[interested_features]

        # Top-1 Motif

        df_motif = get_top_1_motif(
            TimeSeries, num_periods_output, l=5, include_itself=True
        )
        interested_features = [
            c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
        ]
        df_motif = df_motif[interested_features]

        ###
        headers = [
            "traffic",
            "month",
            "day",
            "hour",
            "day_of_week",
            "day_of_year",
            "week_of_year",
        ]
        Data_df = pd.DataFrame(Data, columns=headers)
        sub = Data_df.iloc[:, 1:]
        # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
        New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
        Normalized_Data_df = pd.DataFrame(
            np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
        )
        Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif], axis=1)
    else:
        headers = [
            "traffic",
            "month",
            "day",
            "hour",
            "day_of_week",
            "day_of_year",
            "week_of_year",
        ]
        Data_df = pd.DataFrame(Data, columns=headers)
        sub = Data_df.iloc[:, 1:]
        # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
        New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
        Normalized_Data_df = pd.DataFrame(
            np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
        )
    #################################################################################################
    # cut training and testing training is 10392
    Train = Normalized_Data_df.iloc[0:10392, :]
    Train = Train.values
    Train = Train.astype("float32")
    Test = Normalized_Data_df.iloc[10392 - num_periods_input :, :]
    Test = Test.values
    Test = Test.astype("float32")
    # Number_Of_Features = 7
    Number_Of_Features = len(Normalized_Data_df.columns)
    # print("Number of Features: ", Number_Of_Features)

    ############################################ Windowing ##################################
    end = len(Train)
    start = 0
    next = 0
    x_batches = []
    y_batches = []
    count = 0
    while next + (num_periods_input) < end:
        next = start + num_periods_input
        x_batches.append(Train[start:next, :])
        y_batches.append(Train[next : next + num_periods_output, 0])
        start = start + 1
    y_batches = np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)
    x_batches = np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)

    ############################################ Windowing ##################################
    end_test = len(Test)
    start_test = 0
    next_test = 0
    x_testbatches = []
    y_testbatches = []
    while next_test + (num_periods_input) < end_test:
        next_test = start_test + num_periods_input
        x_testbatches.append(Test[start_test:next_test, :])
        y_testbatches.append(Test[next_test : next_test + num_periods_output, 0])
        start_test = start_test + num_periods_input
    y_testbatches = np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)
    x_testbatches = np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features)

    if include_features and include_motif_information:
        x_batches = x_batches[:, :, :]
        x_testbatches = x_testbatches[:, :, :]
    elif include_features and not include_motif_information:
        x_batches = x_batches[:, :, : (1 + len(headers))]
        x_testbatches = x_testbatches[:, :, : (1 + len(headers))]
    elif not include_features and include_motif_information:
        selected_cols = np.r_[
            0, len(headers) : len(headers) + len(df_motif.columns.tolist())
        ]
        x_batches = x_batches[:, :, selected_cols]
        x_testbatches = x_testbatches[:, :, selected_cols]
    else:
        x_batches = x_batches[:, :, [0]]
        x_testbatches = x_testbatches[:, :, [0]]

    print(
        "len x_testbatches",
        x_testbatches.shape,
        " len y_testbatches",
        y_testbatches.shape,
    )
    return x_batches, y_batches, x_testbatches, y_testbatches

In [18]:
x_batches_Full = []
y_batches_Full = []
X_Test_Full = []
Y_Test_Full = []
# len(data)
for i in range(0, len(data)):
    x_batches = []
    y_batches = []
    X_Test = []
    Y_Test = []
    TimeSeries = data.iloc[i, :]
    x_batches, y_batches, X_Test, Y_Test = New_preprocessing(TimeSeries)
    for element1 in x_batches:
        x_batches_Full.append(element1)

    for element2 in y_batches:
        y_batches_Full.append(element2)

    for element5 in X_Test:
        X_Test_Full.append(element5)

    for element6 in Y_Test:
        Y_Test_Full.append(element6)
    # print("Finished processing time series", i + 1, "of", len(data))

len x_testbatches (7, 24, 7)  len y_testbatches (7, 24, 1)


In [19]:
xgboost_parameters = {
    "learning_rate": 0.2,
    "n_estimators": 800,
    "max_depth": 8,
    "min_child_weight": 1,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 1,
    #  seed=42,silent=False
    "random_state": 42,
    "verbosity": 1,
}

In [20]:
evaluate_with_xgboost(
    num_periods_output,
    x_batches_Full,
    y_batches_Full,
    X_Test_Full,
    Y_Test_Full,
    xgboost_parameters,
)

RMSE:  0.010184501
WAPE:  0.099450774
MAE:  0.00431448


```
average

RMSE:  0.009934292
WAPE:  0.09853438
MAE:  0.0042747236

weighted average

RMSE:  0.009244083
WAPE:  0.10124241
MAE:  0.004392206

direct
RMSE:  0.010279479
WAPE:  0.10846488
MAE:  0.0047055394

top-1
RMSE:  0.009905616
WAPE:  0.105881065
MAE:  0.0045934455

no moitf
RMSE:  0.010184501
WAPE:  0.099450774
MAE:  0.00431448

```

# Results
```
RMSE:  0.009244083
WAPE:  0.10124241
MAE:  0.004392206
```

```
- Top-K Motifs (Average)
	- include_features = True, include_motif_information = True
	- k=3, l=5
	- compute_point_after_average(df_motif)
            RMSE:  0.037377197
            WAPE:  0.3671681
            MAE:  0.019827
	
	- include_features = True, include_motif_information = True
	- k=3, l=5
	- compute_point_after_average(df_motif, method="weighted")
            RMSE:  0.037455145
            WAPE:  0.36559713
            MAE:  0.019742168
	
- Top-K Motifs (Direct)
	- include_features = True, include_motif_information = True
	- k=3, l=5
```


## data= data[0:90,:] (The Whole Dataset)
(We use the (0 0 0 1) for the selection of motif information. The (* * * *) notation is explained in the following part.)
### get_top_k_motif_families
### get_top_k_motifs
- include_features = True
    - get_top_k_motifs(TimeSeries, num_periods_output, k=3, l=10, include_itself=True)
        ```
        RMSE:  0.03711933
        WAPE:  0.3645978
        MAE:  0.019688204
        ```
    - get_top_k_motifs(TimeSeries, num_periods_output, k=2, l=10, include_itself=True)
        ```
        RMSE:  0.037326876
        WAPE:  0.36521178
        MAE:  0.019721359
        ```
- include_features = False
    - get_top_k_motifs(TimeSeries, num_periods_output, k=3, l=10, include_itself=True)
        ```
        RMSE:  0.03767888
        WAPE:  0.37079287
        MAE:  0.020022737
        ```
    - get_top_k_motifs(TimeSeries, num_periods_output, k=2, l=10, include_itself=True)
        ```
        RMSE:  0.037515365
        WAPE:  0.37052742
        MAE:  0.020008402
        ```
### get_top_1_motif
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=24, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.036570087
        WAPE:  0.35921657
        MAE:  0.019397618
        ```
    - include_features = False
        ```
        RMSE:  0.037510373
        WAPE:  0.37156695
        MAE:  0.020064536
        ```
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=20, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.036753196
        WAPE:  0.3616163
        MAE:  0.019527204
        ```
    - include_features = False
        ```
        RMSE:  0.037258912
        WAPE:  0.36707646
        MAE:  0.019822052
        ```
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=10, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.036984496
        WAPE:  0.36259562
        MAE:  0.019580087
        ```
    - include_features = False
        ```
        RMSE:  0.03713597
        WAPE:  0.36674833
        MAE:  0.019804332
        ```
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=5, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.03689488
        WAPE:  0.3659202
        MAE:  0.019759614
        ```
    - include_features = False
        ```
        RMSE:  0.03762679
        WAPE:  0.36732778
        MAE:  0.019835623
        ```
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=3, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.037174594
        WAPE:  0.36547065
        MAE:  0.019735338
        ```
    - include_features = False
        ```
        RMSE:  0.037746847
        WAPE:  0.3686367
        MAE:  0.019906303
        ```
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=2, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.037319317
        WAPE:  0.3677738
        MAE:  0.019859707
        ```
    - include_features = False
        ```
        RMSE:  0.037986483
        WAPE:  0.37716275
        MAE:  0.02036671
        ```
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=1, include_itself=True), No Normalization of Motif Information
    - include_features = True
        ```
        RMSE:  0.03717632
        WAPE:  0.36399755
        MAE:  0.01965579
        ```
    - include_features = False
        ```
        RMSE:  0.03807858
        WAPE:  0.37810946
        MAE:  0.02041783
        ```
- Baseline:
    - include_features = True
        ```
        RMSE:  0.0370458
        WAPE:  0.36180034
        MAE:  0.019537142
        ```
    - include_features = False
        ```
        RMSE:  0.03838941
        WAPE:  0.37899876
        MAE:  0.020465853
        ```

## data= data[0:5,:] (For Preliminary Testing)
- Method used: get_top_1_motif(TimeSeries, num_periods_output, l=1, include_itself=True), No Normalization
    - Without Original Features
        ```
        With Motif Information (0 0 0 1)
        RMSE:  0.03795217
        WAPE:  0.55114305
        MAE:  0.02499656
        
        With Motif Information (0 0 0 0) = (Not include everything)
        RMSE:  0.038168274
        WAPE:  0.55626166
        MAE:  0.02522871
        ```
    - With Original Features
        ```
        With Motif Information (1 1 1 1) = (top_1_motif_dist, top_1_motif_idx, top_1_motif_idx_delta, top_1_motif_point_after_cols)
        RMSE:  0.03793566
        WAPE:  0.5509172
        MAE:  0.024986317

        With Motif Information (1 1 1 0) = (top_1_motif_dist, top_1_motif_idx, top_1_motif_idx_delta)
        RMSE:  0.037920333
        WAPE:  0.55110234
        MAE:  0.024994714

        With Motif Information (1 1 0 1)
        RMSE:  0.037677065
        WAPE:  0.54815537
        MAE:  0.024861056

        With Motif Information (1 1 0 0)
        RMSE:  0.037933104
        WAPE:  0.5537021
        MAE:  0.025112625

        With Motif Information (1 0 1 1)
        RMSE:  0.03787931
        WAPE:  0.5512608
        MAE:  0.025001902

        With Motif Information (1 0 1 0)
        RMSE:  0.037968017
        WAPE:  0.5521225
        MAE:  0.025040982

        With Motif Information (1 0 0 1)
        RMSE:  0.03774264
        WAPE:  0.54919815
        MAE:  0.024908353

        With Motif Information (1 0 0 0)
        RMSE:  0.03787952
        WAPE:  0.5514775
        MAE:  0.02501173

        With Motif Information (0 1 1 1)
        RMSE:  0.03769391
        WAPE:  0.54867697
        MAE:  0.024884714

        With Motif Information (0 1 1 0)
        RMSE:  0.037835274
        WAPE:  0.55371016
        MAE:  0.02511299

        With Motif Information (0 1 0 1)
        RMSE:  0.03787498
        WAPE:  0.55263394
        MAE:  0.02506418

        With Motif Information (0 1 0 0)
        RMSE:  0.03769789
        WAPE:  0.54998827
        MAE:  0.024944188

        With Motif Information (0 0 1 1)
        RMSE:  0.03775543
        WAPE:  0.55012405
        MAE:  0.024950346

        With Motif Information (0 0 1 0)
        RMSE:  0.03769789
        WAPE:  0.54998827
        MAE:  0.024944188

        With Motif Information (0 0 0 1) = (top_1_motif_point_after_cols) <- It is the best.
        RMSE:  0.037602264
        WAPE:  0.5472162
        MAE:  0.024818461

        With Motif Information (0 0 0 0) = (Not include everything)
        RMSE:  0.037865058
        WAPE:  0.54930496
        MAE:  0.024913197
        ```

# References
1. [GBRT-for-TSF/XGBoost_(W-b)/Univariate/xgboostWB_trafffic.py at main · Daniela-Shereen/GBRT-for-TSF](https://github.com/Daniela-Shereen/GBRT-for-TSF/blob/main/XGBoost_(W-b)/Univariate/xgboostWB_trafffic.py)

In [21]:
import datetime

print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")
###
###
###

This notebook was last run end-to-end on: 2025-08-22 17:22:10.781087

